# <center><span style="font-size: 42px;color: darkgreen;">Manutenção e Administração do Hadoop</center></span>

<br><br>

---

<br>

## Desafios na Administração e Manutenção de Cluster Hadoop

<br>

1. **Falta de gestão na configuração**: Configurar um cluster Hadoop exige ajustes específicos para otimizar o desempenho. Sem uma boa gestão, configurações inadequadas podem reduzir a eficiência do sistema e causar problemas operacionais.

2. **Baixa alocação de recursos**: A falta de recursos suficientes (CPU, memória, armazenamento) pode afetar o desempenho do cluster, limitando sua capacidade de processar grandes volumes de dados e escalar conforme necessário.

3. **Gargalos de rede**: Em clusters distribuídos, a rede é crucial. Gargalos na rede podem reduzir a velocidade de transferência de dados entre nós, aumentando o tempo de processamento e criando pontos de lentidão no sistema.

4. **Falta de métricas de monitoramento**: Sem métricas para monitorar a saúde e o desempenho do cluster, é difícil detectar problemas antes que eles afetem a operação. Ferramentas de monitoramento são essenciais para antecipar falhas e otimizar o uso de recursos.

5. **Medidas drásticas para resolver problemas simples**: Em alguns casos, administradores recorrem a soluções extremas, como formatar ou reconstruir o cluster, quando ajustes menores seriam suficientes. Isso pode causar perda de dados e interrupções desnecessárias.

6. **Pontos únicos de falha**: Componentes críticos, como o NameNode, podem representar pontos únicos de falha. Se eles falharem, todo o cluster pode parar. Soluções de alta disponibilidade são necessárias para evitar esses pontos de vulnerabilidade.

7. **Utilização dos valores default para os parâmetros**: Usar configurações padrão para parâmetros de Hadoop pode não ser ideal para todos os ambientes. Ajustar esses parâmetros para as necessidades específicas do cluster é fundamental para garantir um desempenho otimizado.

8. **Falta de profissionais no mercado**: A escassez de profissionais qualificados em Hadoop e Big Data dificulta a administração eficiente dos clusters. Como essas tecnologias exigem conhecimento técnico avançado para configuração, monitoramento e manutenção, a falta de profissionais especializados pode levar a problemas de desempenho, falhas e tempo de inatividade.



<br><br>

---

<br>

## NameNode e Estruturas de Diretórios

<br>

#### Visão Geral

O `NameNode` é o processo master do `HDFS`. Toda as operações de manipulações de arquivos ou pastas do `HDFS` são tratadas pelo `NameNode` pois é ele que contém todas as informações de quais nodes/máquinas ao longo do `Cluster` estão cada seguimento de arquivo salvo no `HDFS`, bem como diretórios existentes e suas permissões de acesso.

#### Estrutura de Funcionamento do NameNode

Os dados no `HDFS` são armazenados em arquivos binários que representam grandes volumes de informações, frequentemente ultrapassando o limite de um único disco rígido. Para superar esse desafio, o `NameNode divide` cada arquivo em blocos de tamanho fixo (tipicamente 128 MB ou 256 MB) e determina quais `DataNodes` receberão esses blocos. Com essa abordagem, o `HDFS` pode distribuir grandes arquivos por vários nós do cluster.

**Importante**: o `NameNode` não armazena nem manipula diretamente os dados. Ele coordena as operações de leitura e gravação, mantendo os metadados, que são dados sobre os dados (como localização dos blocos e permissões de acesso).

##### fsimage e Metadados

O `fsimage` é o arquivo que armazena uma cópia persistente dos metadados do sistema de arquivos no `HDFS`. Esse arquivo contém uma estrutura compacta de todo o sistema de arquivos, incluindo a hierarquia de diretórios, permissões, e a localização de cada bloco em `DataNodes`. O `NameNode` utiliza o `fsimage` em conjunto com o arquivo de log de edição (`edits`) para reconstruir e manter o estado atual dos metadados do sistema de arquivos.

O **`fsimage`** é essencial para a recuperação do sistema, pois fornece uma visão consolidada e otimizada dos metadados. Ao iniciar, o `NameNode` carrega o `fsimage` e aplica as entradas do arquivo `edits` para atualizar o estado até o último ponto de consistência, garantindo que o sistema de arquivos esteja sempre atualizado.

Em **resumo**, o `fsimage` e o `NameNode` trabalham em conjunto para manter a integridade e disponibilidade dos metadados no `HDFS`, enquanto os `DataNodes` armazenam os blocos de dados reais em um formato distribuído e resiliente.

<br><br>

---

<br>

## A Importância do Secondary NameNode

<br>

O `Secondary NameNode` é responsável por sincronizar os arquivos de edição (`edit`) com o `fsimage`, criando uma nova imagem (`fsimage`) mais atualizada. Ele periodicamente combina essas mudanças, evitando que o arquivo `edits` cresça demais e garantindo que o `NameNode` possa restaurar o sistema rapidamente em caso de falhas.

<br><br>

---

## DataNodes e Estrutura de Diretórios

<br>

O `DataNode` é o componente responsável por armazenar os blocos de dados reais, que são acessados pelos usuários (como Cientistas de Dados) em suas aplicações. Cada `DataNode` armazena blocos de dados e se comunica com o `NameNode`, reportando o status dos blocos para que o `NameNode` mantenha uma visão atualizada do sistema.

<br>

####  Operações de Leitura

Durante uma operação de leitura, o `NameNode` indica quais `DataNodes` contêm os blocos solicitados, e o cliente então acessa esses `DataNodes` diretamente para obter os dados. Esse processo garante eficiência, pois o cliente acessa os dados diretamente no `DataNode`, sem passar pelo `NameNode`.

#### Operações de Escrita

Em uma operação de escrita, o cliente primeiro consulta o `NameNode`, que determina quais `DataNodes` receberão os blocos do arquivo. O cliente então envia os dados diretamente para esses `DataNodes`. Cada bloco é replicado em diferentes `DataNodes` conforme o fator de replicação configurado, garantindo a redundância e a resiliência dos dados no cluster.


<br><br>

---

## Metadados do Sistema de Arquivos

<br>

Os **metadados do HDFS** representam a estrutura de diretórios e arquivos no HDFS em uma hierarquia de árvore. Eles incluem atributos fundamentais, como propriedades de diretórios e arquivos, permissões de acesso, cotas de armazenamento (limite de espaço permitido) e o fator de replicação de dados.
<br>

#### O que são Metadados?

Os **metadados** são informações que descrevem o cluster e os dados armazenados no `HDFS`, sem incluir os próprios dados. Eles garantem que o sistema saiba onde e como cada bloco de dados está localizado e configurado.

**Atenção**: Nunca tente modificar manualmente diretórios ou arquivos de metadados, pois isso pode comprometer a integridade do `HDFS` e até causar perda permanente de dados. O **backup dos metadados** é uma tarefa essencial para a segurança e a recuperação de um cluster `Hadoop`.

#### Estrutura dos Metadados

Os **metadados** do sistema são registrados em dois arquivos principais:

- **fsimage**: Armazena uma imagem consolidada do sistema de arquivos `HDFS`, que é usada para inicializar o `NameNode` ao iniciar o sistema. Ele contém uma visão completa da estrutura e das configurações de diretórios e arquivos até o último ponto de consistência.

- **Edit Log**: Durante a operação do `NameNode`, todas as atualizações e alterações de metadados são gravadas continuamente no `Edit Log`. Isso permite que o sistema acompanhe cada modificação realizada no `HDFS` desde a última atualização do `fsimage`.

#### Resumo

O **fsimage** fornece uma base inicial para carregar o `NameNode`, enquanto o **Edit Log** registra as operações subsequentes ao longo do período de operação. Esses dois arquivos trabalham juntos para garantir a integridade e a atualização dos metadados no sistema `HDFS`.

<br><br>

---

## Procedimento de Checkpoint

<br>

O **procedimento de checkpoint** é uma operação crítica no `HDFS` que combina o arquivo `fsimage` e o `Edit Log` para criar uma nova versão consistente do sistema de arquivos. Esse processo é necessário para evitar que o `Edit Log` cresça indefinidamente, o que poderia afetar a performance e a estabilidade do `NameNode`. O **checkpoint** é realizado periodicamente pelo `Secondary NameNode` ou por um `Backup Node` em ambientes de alta disponibilidade.

<br>

#### Como Funciona o Checkpoint

- **Leitura e Carga do fsimage**: O `Secondary NameNode` carrega a versão mais recente do `fsimage` em memória, garantindo que ele tenha uma cópia estável dos metadados até o último ponto de consistência.

- **Aplicação do Edit Log**:: Em seguida, o `Secondary NameNode` lê o `Edit Log` e aplica todas as transações registradas desde o último checkpoint, atualizando a estrutura de metadados em memória para refletir as mudanças mais recentes no sistema de arquivos.

- **Geração de uma Nova Imagem**:: Após aplicar todas as transações, o `Secondary NameNode` cria uma nova versão do `fsimage` com os dados consolidados e armazenados de forma otimizada.

- **Envio ao NameNode**: A nova versão do `fsimage` é enviada de volta ao `NameNode`, substituindo a antiga. O `Edit Log` também é reiniciado, pronto para registrar novas operações. Esse processo mantém o `HDFS` em um estado atualizado e reduz o tamanho do `Edit Log`.

<br>

#### Importância do Checkpoint

O **procedimento de checkpoint** é vital para a manutenção da performance e estabilidade do `HDFS`. Sem **checkpoints** regulares, o `Edit Log` pode se tornar grande demais, tornando o reinício do `NameNode` mais demorado e arriscado, além de consumir mais memória e recursos de processamento. Realizar **checkpoints** em intervalos regulares permite que o sistema se mantenha eficiente, mesmo em ambientes de grande escala.

#### Resumo

O **checkpoint** combina o `fsimage` e o `Edit Log` em um novo estado consolidado do sistema de arquivos, permitindo que o `HDFS` permaneça atualizado e otimizado para o desempenho.


<br><br>

---

## Procedimento de Recuperação à Falha do NameNode

<br>

O **NameNode** é um componente crítico do `HDFS`, responsável por manter os metadados e coordenar o armazenamento de dados no cluster. Em caso de falha do `NameNode`, o acesso ao `HDFS` é interrompido, e o sistema se torna inacessível até que ele seja restaurado.

<br>

#### Passos para Recuperação

- **Reinicialização do NameNode**: Quando ocorre uma falha, o primeiro passo é reiniciar o `NameNode`. Durante essa reinicialização, o `NameNode` carrega o arquivo `fsimage` e aplica o `Edit Log` para restaurar o sistema de arquivos ao último estado consistente.

- **Verificação de Integridade**: Após carregar o fsimage e aplicar as transações do `Edit Log`, o `NameNode` executa verificações de integridade para garantir que os metadados estejam corretos e sem inconsistências.

- **Modo de Segurança**: O `NameNode` entra automaticamente em Modo de Segurança após a inicialização. Durante esse modo, ele verifica se todos os blocos de dados necessários estão disponíveis nos DataNodes antes de permitir operações de leitura e escrita.

- **Reconstrução e Replicação de Blocos**: Se blocos de dados foram perdidos durante a falha, o `HDFS` tenta recriá-los a partir das réplicas disponíveis. Esse processo garante que o sistema possa operar de forma resiliente e completa.

- **Saída do Modo de Segurança**: Uma vez que o `NameNode` tenha validado a integridade dos dados e garantido a disponibilidade dos blocos necessários, ele sai do **Modo de Segurança** e permite que operações normais sejam retomadas no cluster.


<br><br>

---

## Modo de Segurança

<br>

O **Modo de Segurança** é um estado de proteção em que o NameNode entra automaticamente ao ser iniciado. Durante esse estado, o NameNode permanece inativo, permitindo apenas operações de leitura limitada e evitando modificações no sistema de arquivos até que a integridade dos dados seja confirmada.

<br>

#### Características do Modo de Segurança

- **Verificação de Blocos**: No **Modo de Segurança**, o `NameNode` verifica a presença e a integridade de todos os blocos de dados essenciais no cluster. Ele aguarda a confirmação dos `DataNodes` sobre a disponibilidade dos blocos antes de sair desse modo.

- **Replicação de Blocos Perdidos**: Se algum bloco estiver faltando, o `NameNode` inicia o processo de replicação usando as réplicas disponíveis nos `DataNodes`. Essa replicação assegura que o cluster esteja em um estado consistente antes de sair do Modo de Segurança.

- **Monitoramento de Replicação**: O Modo de Segurança permite ao `NameNode` garantir que o fator de replicação seja atendido para cada bloco, assegurando a redundância dos dados conforme configurado.

- **Desbloqueio Manual**: Em casos de recuperação demorada ou se um administrador precisar realizar verificações adicionais, o Modo de Segurança pode ser desativado manualmente usando o comando `hdfs dfsadmin -safemode leave`. Entretanto, essa ação deve ser tomada com cautela, pois pode impactar a consistência dos dados.

<br><br>

---

## Backup no Cluster Hadoop

<br>

Realizar **backups regulares** No `Hadoop` é essencial para proteger os dados e garantir a continuidade do sistema em casos de falhas. No `Hadoop`, o backup abrange duas áreas principais: **backup dos metadados** e **backup dos dados**.

<br>

### Estragégias de Backup do HDFS

#### Backup dos Metadados

Os metadados do `HDFS` incluem informações sobre a estrutura de diretórios e arquivos, permissões e a localização dos blocos nos DataNodes. Fazer backup dos metadados é essencial para que o `NameNode` possa reconstruir a estrutura do sistema de arquivos em caso de falha. Duas abordagens principais para backup de metadados são:

- **Checkpoint do Secondary NameNode**: O `Secondary NameNode` realiza checkpoints periódicos, combinando o fsimage (imagem dos metadados) com o `Edit Log` (log de operações). Isso gera uma nova versão do `fsimage`, garantindo que o `NameNode` tenha um estado atualizado dos metadados.

- **Snapshots de Metadados**: Além dos checkpoints, snapshots de metadados podem ser armazenados para recuperação rápida em situações críticas. Ferramentas como o `HDFS Snapshot` são úteis para criar e gerenciar esses backups.

#### Backup dos Dados

O backup dos dados no `HDFS` protege os arquivos propriamente ditos e garante sua disponibilidade em casos de perda. Há duas abordagens principais para realizar esse backup:

- **Replicação entre Clusters**: A replicação dos dados para outro cluster de backup é uma solução robusta, especialmente em cenários de alta disponibilidade. Ferramentas como o `DistCp` facilitam essa replicação, copiando dados de maneira automatizada entre clusters diferentes.

- **Armazenamento Externo**: Utilizar sistemas de armazenamento externo, como `NAS` ou `AWS S3`, oferece uma camada adicional de segurança e durabilidade, especialmente para grandes volumes de dados. Esses sistemas são ideais para armazenamento de longo prazo e recuperação em caso de desastres.

<br>

#### Apenas o Backup é suficiente?

- **Divisão dos dados em críticos e não críticos**: Identificar quais dados são essenciais (críticos) e quais não são (não críticos) permite priorizar o armazenamento seguro e as estratégias de backup.
- **Formato do storage que será usado com o HDFS**: Escolher o formato de armazenamento ideal para o HDFS é fundamental para garantir a compatibilidade, desempenho e segurança dos dados. Opções de storage como `NAS` e `S3` são comuns em backups Hadoop. No entanto, o formato deve ser otimizado para suportar grandes volumes e garantir velocidade nas operações de recuperação.
- **Monitoramento do Cluste**: Implementar um `sistema de monitoramento contínuo` permite detectar e resolver problemas antes que eles causem falhas críticas. 
- **Aplicação de Patches e Correções de Segurança**: Manter o cluster atualizado com patches de segurança e correções garante proteção contra vulnerabilidades que podem comprometer a integridade do sistema.

<br>

#### Comandos Para Verificação de Falhas

- **Filesystem check**: `hdfs fsck /`
- **Filesystem Balancer**: `hdfs balancer`
- **Relatório do cluster**: `hdfs dfsadmin -report`

<br><br>

---

## Solução de Problemas no Cluster Hadoop

<br>

#### Principais Problemas na Administração do Hadoop

- **Erro Humano**: A administração manual pode introduzir erros, uma das causas mais comuns de problemas no `Hadoop`. Configurações inadequadas, execução de comandos incorretos e outros descuidos podem comprometer o funcionamento do cluster.
- **Hardware**: Falhas de discos ocorrem com frequência e nem sempre uma vez. A degradação ao longo do tempo, pode levar à lentidão, antes de uma falha acontecer.
- **Erros de Configuração**: É difícil configurar um ambiente `Hadoop` 100% otimizado e são muitos parâmetros possíveis. A utilização destes parâmetros depende de uma série de fatores e por isso recomenda-se a realização maciça de testes antes de aplicar alterações em produção.
- **Excessiva Utilização de Recursos**: Erros em tarefas devem ser investigados e resolvidos. Erros recorrentes, consomem recursos do servidor, degradando performance
- **Identificação dos Nodes**: Configuração de rede dos nodes é um item crítico, pois a comunicação entre eles poderá gerar problemas de performance.

<br>

#### Principais Soluções Para Problemas no Cluster

- **Auditoria do Ambiente**: Realize verificações regulares no ambiente, analisando o histórico de operação do sistema. Identificar atividades anômalas ou padrões recorrentes é essencial para manter o desempenho e a segurança do cluster.

- **Verificação de Dependências**: O funcionamento adequado do MapReduce, por exemplo, depende do HDFS estar em plena operação. Sempre verifique se todos os serviços e componentes estão ativos e funcionando corretamente antes de iniciar ou depurar tarefas dependentes.

- **Identificação de Padrões de Falhas**: Tente reproduzir os erros observados para identificar padrões e causas. Isso facilita o diagnóstico e pode revelar falhas estruturais ou de configuração.

- **Verificação de Logs**: Consulte os logs de todos os componentes (`NameNode`, `DataNode`, `ResourceManager`, etc.) para identificar a origem dos problemas. Os logs fornecem detalhes valiosos para entender falhas e otimizar o sistema.

- **Gerenciamento de Recursos**: Certifique-se de que há espaço em disco e memória suficientes para todos os serviços e `daemons do Hadoop` operarem corretamente. A falta de recursos pode causar interrupções e falhas.

- **Monitoramento de Eventos de Falha**: Acompanhe o fluxo de eventos que levam a falhas, identificando dependências e ações específicas que desencadeiam problemas. Isso permite antecipar falhas e implementar medidas preventivas.

<br><br>

---

## Autenticação e Segurança Hadoop

<br>

A **segurança no Hadoop** é essencial para proteger os dados e assegurar que apenas usuários e sistemas autorizados possam acessar e manipular informações no cluster. Implementar mecanismos de autenticação e segurança adequados ajuda a prevenir acesso não autorizado, garantir a integridade dos dados e proteger o ambiente contra vulnerabilidades.

<br>

#### Mecanismos de Autenticação e Segurança

- **Autenticação de Usuários**: O `Hadoop` utiliza métodos de autenticação para garantir que apenas usuários verificados possam acessar o cluster. Entre os mecanismos mais comuns estão o uso de `Kerberos` e outras práticas de autenticação integradas ao ambiente `Hadoop`.

- **Controle de Acesso baseado em Permissões**: O `Hadoop` permite o controle de acesso a arquivos e diretórios através de permissões, definindo quem pode ler, escrever ou executar cada arquivo no sistema. Isso ajuda a restringir o acesso e manter a segurança dos dados.

- **Encriptação de Dados**: Tanto os dados em trânsito quanto os armazenados podem ser criptografados para evitar que sejam acessados por pessoas não autorizadas. A encriptação garante que dados sensíveis permaneçam protegidos, mesmo em caso de vazamento.

- **Firewalls e Redes Seguras**: O uso de firewalls e a configuração de redes seguras ajuda a proteger o cluster `Hadoop` contra acessos externos indesejados, restringindo o tráfego e garantindo que apenas conexões seguras cheguem ao cluster.

<br>

### Protocolo Kerberos

O **Kerberos** é o protocolo de autenticação mais comum no Hadoop, amplamente utilizado em clusters distribuídos para garantir a segurança e autenticação dos usuários e serviços. Ele oferece um sistema de autenticação robusto, utilizando um mecanismo de "tickets" para confirmar identidades de usuários e serviços sem a necessidade de compartilhar senhas constantemente.

<br>

#### Como o Kerberos Funciona no Hadoop

- **Autenticação Inicial**: O usuário ou serviço solicita um "ticket" ao servidor de autenticação (`KDC - Key Distribution Center`) do Kerberos, provando sua identidade através de credenciais seguras.

- **Recebimento do Ticket**: Após autenticar o usuário ou serviço, o `KDC` emite um "Ticket-Granting Ticket" (TGT), que o usuário pode usar para solicitar acesso a serviços específicos no cluster.

- **Acesso aos Serviços**: Com o TGT em mãos, o usuário pode solicitar ao KDC acesso a serviços específicos do `Hadoop`, como o `NameNode` ou `ResourceManager`. O Kerberos emite um ticket específico para cada serviço solicitado.

- **Validação e Comunicação Segura**: O ticket específico permite que o usuário acesse o serviço desejado de forma segura e autenticada. O Kerberos gerencia a validade desses tickets, renovando ou expirando-os conforme necessário para manter a segurança.

#### Benefícios do Kerberos no Hadoop

- **Segurança e Confidencialidade**: O Kerberos oferece um nível de segurança elevado, exigindo autenticação de usuários e serviços. Isso protege o cluster contra ataques de "spoofing" e acesso não autorizado.

- **Autenticação Centralizada**: O Kerberos centraliza o gerenciamento de autenticação, simplificando o controle de usuários e permitindo a aplicação de políticas de segurança consistentes em todo o cluster.

- **Integração com Hadoop**: O Kerberos é nativamente suportado pelo `Hadoop`, tornando-o uma escolha eficaz para a autenticação e proteção do ambiente.


#### Configuração do Kerberos no Hadoop

Para configurar o Kerberos no Hadoop, siga os passos descritos abaixo:

- 1. **Instalar os pacotes do Kerberos no sistema operacional**:

  - Utilize o comando para instalação dos pacotes essenciais:
```code
yum install krb5-server krb5-libs krb5-auth-dialog krb5-workstation
```
<br>

- 2. **Editar os arquivos de configuração**:

  - Modifique os seguintes arquivos para definir o servidor e as políticas de autenticação do Kerberos:
    - `/etc/krb5.conf` – arquivo de configuração principal do cliente Kerberos.
    - `/var/kerberos/krb5kdc/kdc.conf` – arquivo de configuração do servidor KDC.

<br>

- 3. **Copiar as versões atualizadas dos arquivos para cada node no cluster**:

  - Após configurar e validar os arquivos de configuração, copie-os para todos os nodes do cluster para assegurar uma configuração de Kerberos consistente em todo o ambiente distribuído.

<br><br>

---

<br><br>
<br>

# Fim!